<a href="https://colab.research.google.com/github/advik-7/Court_Room_Simulation/blob/main/Court_Room_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_community -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00


In [3]:

!pip install pypdf -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.6 MB/s eta 0:00:00


In [4]:
!pip install --upgrade google-generativeai -qq

!pip install faiss-cpu==1.7.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 40.4 MB/s eta 0:00:00


In [6]:
!pip install pypdf

In [7]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool
from langchain.llms.base import LLM
from langchain.vectorstores import FAISS
import requests
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import google.generativeai as genai
import os
from typing import Optional, List, Mapping, Any

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA



In [63]:
def create_faiss_db(all_documents, model_name, faiss_index_path, text_file_path):
    all_documents = []
    for file_path in file_paths:
        loader = PyPDFLoader(file_path)
        all_documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1250, chunk_overlap=0)
    texts = text_splitter.split_documents(all_documents)

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode([t.page_content for t in texts])

    class CustomEmbeddings:
        def __init__(self, model):
            self.model = model

        def embed_documents(self, texts):
            return self.model.encode(texts)

        def embed_query(self, text):
            return self.model.encode(text)

        def __call__(self, text):
            return self.model.encode(text)

    custom_embeddings = CustomEmbeddings(model)

    db = FAISS.from_embeddings(
        text_embeddings=[(t.page_content, embedding) for t, embedding in zip(texts, embeddings)],
        embedding=custom_embeddings
    )

    faiss.write_index(db.index, faiss_index_path)

    with open(text_file_path, "w", encoding="utf-8") as file:
        for chunk in [t.page_content for t in texts]:
            file.write(chunk + "\n")
@tool
def retrieve_relevant_chunks(query, faiss_index_path, text_file_path, model_name, top_k=5):
    """Generalized function to retrieve relevant chunks using FAISS."""
    index = faiss.read_index(faiss_index_path)
    with open(text_file_path, "r", encoding="utf-8") as file:
        chunks = file.readlines()

    model = SentenceTransformer(model_name)
    query_embedding = model.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i].strip() for i in indices[0]]


@tool
def query_llm(prompt: str, context: str = "") -> str:
    """LLM query tool for response generation using Gemini."""
    time.sleep(3)
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    model = genai.GenerativeModel("gemini-1.5-flash")

    legal_advisor_prompt = """
    You are a legal expert tasked with providing insightful analysis and guidance on legal matters. Your role is to objectively assess situations, identify relevant laws and precedents, and present well-reasoned arguments based on the information provided.

    Instructions:

    1. Legal Analysis: Carefully analyze the context and the specific question presented. Identify applicable laws, regulations, and legal precedents that are relevant to the situation.
    2. Objective Reasoning: Present your analysis in an objective and unbiased manner. Consider different legal perspectives and potential arguments from all sides.
    3. Clear and Concise Communication: Clearly articulate your findings, conclusions, and recommendations. Support your points with legal citations and logical reasoning.

    Example Scenarios:

    User: "What are the legal implications of this contract clause?"
    Response: "This clause appears to be governed by contract law principles related to... Relevant case law includes... This could potentially lead to..."

    User: "Is this action likely to be considered negligence?"
    Response: "To determine negligence, we need to consider the elements of duty, breach, causation, and damages. Based on the provided context..."

    Remember: Your primary goal is to provide accurate and insightful legal analysis based on the information given.
    """


    complete_prompt = f"{legal_advisor_prompt}\n\n{context}\n\nQuestion: {prompt}\nAnswer:"
    response = model.generate_content([complete_prompt])
    return response.text

In [55]:
def run_agentic_rag(query, faiss_index_path, text_file_path, defense_argument=None):
    """Runs the agentic RAG pipeline to generate the prosecutor's points.

    Args:
        query: The initial query or scenario.
        faiss_index_path: Path to the FAISS index.
        text_file_path: Path to the text file containing chunks.
        model_name: Name of the SentenceTransformer model.
        defense_argument: Optional argument from the defense lawyer.

    Returns:
        prosecutors_point: The prosecutor's argument.
    """
    tools = [
        Tool(
            name="Retrieve",
            func=lambda q: retrieve_relevant_chunks(q, faiss_index_path, text_file_path, model_name),
            description="Useful for retrieving relevant chunks of information from the legal document."
        ),
        Tool(
            name="Analyze",
            func=query_llm,
            description="Useful for analyzing the situation, identifying applicable laws, and presenting arguments as a prosecutor."
        )
    ]

    class CustomGemini:
        """Custom class to interact with Google Gemini."""

        def __init__(self, temperature: float, max_tokens: int, model: str, google_api_key: str):
            self.temperature = temperature
            self.max_tokens = max_tokens
            self.model = model
            genai.configure(api_key=google_api_key)
            self.generation_config = {
                "temperature": temperature,
                "max_output_tokens": max_tokens,
            }
            self.model_instance = genai.GenerativeModel(
                model_name=model,
                generation_config=self.generation_config,
            )

        def __call__(self, prompt: str, stop: Optional[List[str]] = None) -> str:
            response = self.model_instance.generate_content([prompt])
            return response.text
    custom_gemini = CustomGemini(
        temperature=0.7,
        max_tokens=512,
        model="gemini-1.5-flash",
        google_api_key=os.environ["GEMINI_API_KEY"]
    )


    class CustomLLMWrapper(LLM):
        custom_llm: CustomGemini

        @property
        def _llm_type(self) -> str:
            return "custom_gemini"

        def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
            return self.custom_llm(prompt, stop=stop)

        @property
        def _identifying_params(self) -> Mapping[str, Any]:
            return {
                "temperature": self.custom_llm.temperature,
                "max_tokens": self.custom_llm.max_tokens,
                "model": self.custom_llm.model,
            }


    wrapped_llm = CustomLLMWrapper(custom_llm=custom_gemini)


    agent = initialize_agent(
        tools=tools,
        llm=wrapped_llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        output_parser=parser,
        handle_parsing_errors=True
    )

    prosecutor_prompt = f"""
    You are a prosecutor presenting your case before a judge.
    Your objective is to prove the defendant's guilt beyond a reasonable doubt.
    Analyze the situation described in the query and the provided context.
    Present clear and concise arguments, supporting your claims with evidence and legal reasoning.
    Address the judge directly as "Your Honor."

    If the defense lawyer has presented an argument, directly refute it with counter-evidence and legal precedents.

    Example:
    "Your Honor, the evidence clearly shows that the defendant was at the scene of the crime..."
    """

    # Incorporate defense argument if provided
    if defense_argument:
        prosecutor_prompt += f"""
    The defense lawyer has presented the following argument: {defense_argument}
    Refute this argument with evidence and legal reasoning.
    """

    prosecutors_point = agent.run(prosecutor_prompt + query)  # Store the argument
    return prosecutors_point

In [56]:
def run_defense_rag(query, faiss_index_path, text_file_path, prosecution_argument=None):
    """Runs the agentic RAG pipeline to generate the defense lawyer's argument.

    Args:
        query: The initial query or scenario.
        faiss_index_path: Path to the FAISS index.
        text_file_path: Path to the text file containing chunks.
        model_name: Name of the SentenceTransformer model.
        prosecution_argument: Optional argument from the prosecutor.

    Returns:
        defense_argument: The defense lawyer's argument.
    """
    tools = [
        Tool(
            name="Retrieve",
            func=lambda q: retrieve_relevant_chunks(q, faiss_index_path, text_file_path, model_name),
            description="Useful for retrieving relevant chunks of information from the legal document."
        ),
        Tool(
            name="Analyze",
            func=query_llm,  # Assuming query_llm is your generic legal analysis tool
            description="Useful for analyzing the situation, identifying applicable laws, and presenting arguments as a defense lawyer."
        )
    ]


    class CustomGemini:
        """Custom class to interact with Google Gemini."""

        def __init__(self, temperature: float, max_tokens: int, model: str, google_api_key: str):
            self.temperature = temperature
            self.max_tokens = max_tokens
            self.model = model
            genai.configure(api_key=google_api_key)
            self.generation_config = {
                "temperature": temperature,
                "max_output_tokens": max_tokens,
            }
            self.model_instance = genai.GenerativeModel(
                model_name=model,
                generation_config=self.generation_config,
            )

        def __call__(self, prompt: str, stop: Optional[List[str]] = None) -> str:
            response = self.model_instance.generate_content([prompt])
            return response.text
    custom_gemini = CustomGemini(
        temperature=0.7,
        max_tokens=512,
        model="gemini-1.5-flash",
        google_api_key=os.environ["GEMINI_API_KEY"]
    )


    class CustomLLMWrapper(LLM):
        custom_llm: CustomGemini

        @property
        def _llm_type(self) -> str:
            return "custom_gemini"

        def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
            return self.custom_llm(prompt, stop=stop)

        @property
        def _identifying_params(self) -> Mapping[str, Any]:
            return {
                "temperature": self.custom_llm.temperature,
                "max_tokens": self.custom_llm.max_tokens,
                "model": self.custom_llm.model,
            }


    wrapped_llm = CustomLLMWrapper(custom_llm=custom_gemini)


    agent = initialize_agent(
        tools=tools,
        llm=wrapped_llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        output_parser=parser,
        handle_parsing_errors=True
    )
    defense_prompt = f"""
    You are a defense lawyer presenting your case before a judge.
    Your objective is to defend your client and cast doubt on the prosecution's case.
    Analyze the situation described in the query and the provided context.
    Present strong arguments to protect your client's rights and challenge the prosecution's claims.
    Address the judge directly as "Your Honor."

    If the prosecutor has presented an argument, directly refute it with counter-evidence and legal precedents.

    Example:
    "Your Honor, the prosecution's evidence is circumstantial and does not conclusively prove my client's guilt..."
    """

    if prosecution_argument:
        defense_prompt += f"""
    The prosecutor has presented the following argument: {prosecution_argument}
    Refute this argument with evidence and legal reasoning.
    """

    defense_argument = agent.run(defense_prompt + query)
    return defense_argument

In [57]:
arguments_history = []
import time
def run_legal_battle(query, faiss_index_path, text_file_path, num_rounds=3):
    prosecutor_argument = None
    defense_argument = None

    for _ in range(num_rounds):
        prosecutor_argument = run_agentic_rag(query, faiss_index_path, text_file_path, defense_argument)
        arguments_history.append(("Prosecutor", prosecutor_argument))
        time.sleep(wait_time)
        defense_argument = run_defense_rag(query, faiss_index_path, text_file_path, prosecutor_argument)
        arguments_history.append(("Defense", defense_argument))
        time.sleep(wait_time)  # Wait after defense's turn

    return arguments_history

In [58]:
from concurrent.futures import wait
wait_time=5

In [59]:
from langchain_core.exceptions import OutputParserException  # Assuming langchain_core is used
from langchain_core.output_parsers import BaseOutputParser

class PassthroughOutputParser(BaseOutputParser[str]):  # Inherit from BaseOutputParser
    """Custom output parser that simply returns the raw text."""

    def parse(self, text: str) -> str:
        """Return the raw text as is."""
        return text

    @property
    def _type(self) -> str:
        return "passthrough_output_parser"

In [13]:
model_name="sentence-transformers/LaBSE"

In [14]:
parser = PassthroughOutputParser()

In [61]:
faiss_index_path="/content/law_faiss_index.bin"
text_file_path="/content/law_texts3.txt"
file_paths=["/content/JN PandeyThe-Constitutional-Law-Of-India-Edition-_241219_203828.pdf"]
create_faiss_db(file_paths,model_name,faiss_index_path,text_file_path)

In [15]:
faiss_index_path="/content/law_faiss_index.bin"
text_file_path="/content/law_texts3.txt"

In [64]:

query='''Mr. Arun, an entrepreneur, set up a factory in a rural area and hired only local laborers to promote local employment. He justified his decision under Article 39 of the Indian Constitution, which directs the State to promote equal opportunity and protect weaker sections of society. However, a qualified engineer, Ms. Priya, who was not a local resident, applied for a job but was denied employment based on her place of residence. She sued Mr. Arun, arguing that the decision violated her fundamental right to equality under Article 14 and right to protection against discrimination under Article 16(2).'''
run_legal_battle(query, faiss_index_path, text_file_path)



> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Thought: I need to analyze the situation and formulate arguments to prove Mr. Arun's guilt beyond a reasonable doubt.  I will focus on refuting Mr. Arun's justification using Article 39 and demonstrating a violation of Articles 14 and 16(2).

Action: Analyze
Action Input: Mr. Arun, an entrepreneur, set up a factory in a rural area and hired only local laborers to promote local employment. He justified his decision under Article 39 of the Indian Constitution, which directs the State to promote equal opportunity and protect weaker sections of society. However, a qualified engineer, Ms. Priya, who was not a local resident, applied for a job but was denied employment based on her place of residence. She sued Mr. Arun, arguing that the decision violated her fundamental right to equality under Article 14 and right to protection against discrimination under Article 16(2).

Observa

[('Prosecutor',
  "Your Honor, the prosecution presents its case against Mr. Arun.  While Mr. Arun claims his actions were justified under Article 39 of the Indian Constitution to promote local employment, his actions constitute a clear violation of Ms. Priya's fundamental rights under Articles 14 and 16(2).\n\nAs detailed in the submitted legal argument,  Mr. Arun's discriminatory hiring practice, based solely on Ms. Priya's non-local residence, violates Article 14's guarantee of equality before the law and equal protection.  This arbitrary classification lacks a rational nexus to the job requirements.  The state's directive under Article 39 does not authorize private individuals to discriminate.  Furthermore, the principle of non-discrimination in employment, while explicitly addressed in Article 16(2) for government employment, extends in spirit to private employment concerning fundamental rights.  Discriminating based on residence is a clear instance of prohibited discrimination, i

In [65]:
for i, (speaker, argument) in enumerate(arguments_history):
    print(f"{i+1}. {speaker}: {argument}")

1. Prosecutor: Your Honor, the prosecution presents its case against Mr. Arun.  While Mr. Arun claims his actions were justified under Article 39 of the Indian Constitution to promote local employment, his actions constitute a clear violation of Ms. Priya's fundamental rights under Articles 14 and 16(2).

As detailed in the submitted legal argument,  Mr. Arun's discriminatory hiring practice, based solely on Ms. Priya's non-local residence, violates Article 14's guarantee of equality before the law and equal protection.  This arbitrary classification lacks a rational nexus to the job requirements.  The state's directive under Article 39 does not authorize private individuals to discriminate.  Furthermore, the principle of non-discrimination in employment, while explicitly addressed in Article 16(2) for government employment, extends in spirit to private employment concerning fundamental rights.  Discriminating based on residence is a clear instance of prohibited discrimination, infring